In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
!mkdir -p pipelines/code

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
%%writefile pipelines/code/user_item_matrix_transformer.py

import numpy as np
import pandas as pd

from pipeliner.recommendations.transformer import UserItemMatrixTransformer

data_types = {"user_id": str, "item_id": str, "rating": np.float64}

if __name__ == "__main__":
    base_dir = "/opt/ml/processing"
    input_file = "user_item_ratings.csv"
    output_file = "user_item_matrix.csv"

    user_item_ratings = pd.read_csv(f"{base_dir}/{input_file}", dtype=data_types)
    transformer = UserItemMatrixTransformer()
    user_item_matrix = transformer.transform(user_item_ratings)

    user_item_matrix.to_csv(f"{base_dir}/{output_file}", header=True, index=False)

In [ ]:
%%writefile pipelines/code/similarity_matrix_transformer.py

import numpy as np
import pandas as pd

from pipeliner.recommendations.transformer import SimilarityTransformer

import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--kind", type=str, default="user")
parser.add_argument("--metric", type=str, default="cosine")
args = parser.parse_args()

if __name__ == "__main__":
    base_dir = "/opt/ml/processing"
    input_file = "user_item_matrix.csv"
    output_file = "item_similarity_matrix.csv"

    user_item_matrix = pd.read_csv(f"{base_dir}/{input_file}", dtype=np.float64)
    transformer = SimilarityTransformer(kind=args.kind, metric=args.metric)
    similarity_matrix = transformer.transform(user_item_matrix)

    similarity_matrix.to_csv(f"{base_dir}/{output_file}", header=True, index=False)

In [ ]:
%%writefile pipelines/code/item_recommender_train.py



In [ ]:
from sagemaker.workflow.pipeline_context import LocalPipelineSession
from pipeliner.recommendations.pipelines import RecommenderPipeline

session = LocalPipelineSession()
pipeline = RecommenderPipeline().create(
    role="local", name="recommender", session=session
)

pipeline.start()